# Модели на предобученных сетях ru-bert sbert

### импорт пакетов

In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
import matplotlib.pyplot as plt
from pandas import DataFrame
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
import multiprocessing
from numpy import save, load

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, f1_score, cohen_kappa_score, roc_auc_score
from sklearn import utils
from sklearn import linear_model
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

In [3]:
import nltk
import pymystem3
import gensim
import pymorphy2
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import SnowballStemmer

In [4]:
import transformers
import torch
from typing import Tuple, List
from functools import partial
from torch.utils.data import Dataset, DataLoader, RandomSampler
from torch.nn.utils.rnn import pad_sequence
import torch.nn as nn
import torch.optim as optim
from transformers import AutoTokenizer, AutoModel

In [5]:
from transformers import get_linear_schedule_with_warmup, AdamW

In [6]:
from collections import defaultdict

In [7]:
multihot_batch = torch.tensor([[0,1,0,0], [0,0,0,1], [0,0,1,0]])
vnon = (multihot_batch == torch.tensor(1)).nonzero(as_tuple=False)
v0 = vnon[:,0]
v1 = vnon[:,1]
split_ind = ((torch.roll(v0, -1, 0) - v0) == 1).nonzero(as_tuple=False)[:,0] + 1
split_size = torch.cat([split_ind[0].view(1),(torch.roll(split_ind, -1, 0) - split_ind)[:-1]])
final_size = torch.tensor([torch.numel(v1) - torch.sum(split_size)])
split_size = torch.cat([split_size, final_size])


### добавление русских стемов и лем

In [8]:
morph = pymorphy2.MorphAnalyzer()

In [9]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/stanislavilusin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/stanislavilusin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [11]:
mystem = Mystem() 
russian_stopwords = stopwords.words("russian")

### добавление числительных на сумм конракта для удаления из текста

In [12]:
russian_stopwords.extend(['один','два',"три","четыре","пять","шесть","семь","восемь","девять","сто","двести",
                          "триста","четыреста","пятьсот","шестьсот","восемьсот","девятьсот","миллион","рубль","копейка",
                         'семьсот',"десять","двадцать","тридцать","сорок","пятьдесят","шестьдесят","семьдесят","восемьдесят",
                          "девяносто","тысяча","одиннадцать","двенадцать","тринадцать","четырнадцать","пятнадцать",
                          "шестнадцать","семнадцать","восемнадцать", "девятнадцать"])

### функция для обработки (лемматизация токенизация удаления эмотиконов токенизация из пакетов нлтк и возвращение начальной формы токена)

In [13]:
def preprocess_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [morph.parse(token)[0].normal_form for token in tokens]
    tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " " \
              and token.strip() not in punctuation]
    
    text = " ".join(tokens)
    
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=|_|__|___)(?:-)?(?:\)|\(|D|P)', text)
    text = (re.sub('[\W]+', ' ', text.lower())+ ' '.join(emoticons).replace('-', '')).replace('_', '')
    
    tokens = []
    for sent in nltk.sent_tokenize(text, language = 'russian'):
        for word in nltk.word_tokenize(sent, language = 'russian'):
            if len(word) <= 3:
                continue
            word = morph.parse(word.lower())[0].normal_form
            tokens.append(word.lower())
    return tokens


## Модель на ru-bert tiny (предобученная классификация на основе модельных эмбеддингов по токенам)

### функция возвращения токен-слой к текст-слой через маску последнего скрытого слоя берта

### скачиваем токенизатор и модель 

### загрузка данных для трейна и теста

In [14]:
data = pd.read_excel(r'model_out_total.xlsx')
data = data[['Шаблон', 'Флаг']]

### приведение данных к очищенному стандарту, также объединение токенов в исходные предложение по предположению обученной модели

In [15]:
for index, row in data.iterrows():
    data['Шаблон'].loc[index] = ' '.join(preprocess_text(row['Шаблон']))

### скачиваем предобученный токенизатор и модель

In [16]:
tokenizer_sber = AutoTokenizer.from_pretrained("sberbank-ai/sbert_large_nlu_ru")
model_sber = AutoModel.from_pretrained("sberbank-ai/sbert_large_nlu_ru")

In [17]:
data_tr = pd.get_dummies(data, columns = ['Флаг'])

In [18]:
data_tr, data_val = train_test_split(data_tr, test_size=0.2, random_state=2022, stratify  = data['Флаг'])

In [19]:
device = torch.device('cpu')
if torch.cuda.is_available():
    device = torch.device('cuda:0')

BATCH_SIZE = 16
train_dataset = TorchDataset(tokenizer_sber, data_tr, lazy=True)
dev_dataset = TorchDataset(tokenizer_sber, data_val, lazy=True)
train_iterator = DataLoader(train_dataset, batch_size=BATCH_SIZE)#, sampler=train_sampler, collate_fn=collate_fn)
dev_iterator = DataLoader(dev_dataset, batch_size=BATCH_SIZE)#, sampler=dev_sampler, collate_fn=collate_fn)

In [20]:
class ToxicDataset(Dataset):
    
    def __init__(self, tokenizer: tokenizer_sber, dataframe: pd.DataFrame, lazy: bool = False):
        self.tokenizer = tokenizer
        self.pad_idx = tokenizer.pad_token_id
        self.lazy = lazy
        if not self.lazy:
            self.X = []
            self.Y = []
            for i, (row) in tqdm(dataframe.iterrows()):
                x, y = self.row_to_tensor(self.tokenizer, row)
                self.X.append(x)
                self.Y.append(y)
        else:
            self.df = dataframe        
    
    @staticmethod
    def row_to_tensor(tokenizer: tokenizer_sber, row: pd.Series) -> Tuple[torch.LongTensor, torch.LongTensor]:
        tokens = tokenizer.encode(row["Шаблон"], add_special_tokens=True)
        if len(tokens) > 256:
            tokens = tokens[:255] + [tokens[-1]]
        x = torch.LongTensor(tokens)
        y = torch.FloatTensor(row[['Флаг_постоплата', 'Флаг_предоплата',
       'Флаг_помесячно']])
        return x, y
        
    
    def __len__(self):
        if self.lazy:
            return len(self.df)
        else:
            return len(self.X)

    def __getitem__(self, index: int) -> Tuple[torch.LongTensor, torch.LongTensor]:
        if not self.lazy:
            return self.X[index], self.Y[index]
        else:
            return self.row_to_tensor(self.tokenizer, self.df.iloc[index])
            

def collate_fn(batch: List[Tuple[torch.LongTensor, torch.LongTensor]], device: torch.device) \
        -> Tuple[torch.LongTensor, torch.LongTensor]:
    x, y = list(zip(*batch))
    x = pad_sequence(x, batch_first=True, padding_value=0)
    y = torch.stack(y)
    return x.to(device), y.to(device)

train_dataset = ToxicDataset(tokenizer_sber, data_tr, lazy= True)
dev_dataset = ToxicDataset(tokenizer_sber, data_val, lazy= True)
collate_fn = partial(collate_fn, device=device)
BATCH_SIZE = 16
train_sampler = RandomSampler(train_dataset)
dev_sampler = RandomSampler(dev_dataset)
train_iterator = DataLoader(train_dataset, batch_size=BATCH_SIZE, sampler=train_sampler, collate_fn=collate_fn)
dev_iterator = DataLoader(dev_dataset, batch_size=BATCH_SIZE, sampler=dev_sampler, collate_fn=collate_fn)

In [23]:
from sklearn.utils.class_weight import compute_class_weight

class_wts = compute_class_weight(class_weight = 'balanced', classes = np.unique(data.loc[data_tr.index]['Флаг']), y = data.loc[data_tr.index]['Флаг'])

print(class_wts)

[1.50147493 0.84411277 0.87008547]


In [24]:
weights= torch.tensor(class_wts,dtype=torch.float)
weights = weights.to(device)
criterion = nn.NLLLoss(weight=weights)

In [25]:
class BertClassifier(nn.Module):
    
    def __init__(self, bert: model_sber, num_classes: int):
        super().__init__()
        self.bert = bert
        self.dropout = nn.Dropout(0.1)
        self.classifier1 = nn.Linear(bert.config.hidden_size, 256) #256
        self.classifier2 = nn.Linear(256,num_classes) #256
        self.relu = nn.ReLU()
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, input_ids, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None, labels=None):
        outputs = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids,
        position_ids=position_ids, head_mask=head_mask)
        cls_output = outputs[1] # batch, hidden
        x = self.classifier1(cls_output)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.classifier2(x)
        cls_output = self.softmax(x)
    
        loss = 0
        if labels is not None:
            multihot_batch = labels
            vnon = (multihot_batch == torch.tensor(1)).nonzero(as_tuple=False)
            v0 = vnon[:,0]
            v1 = vnon[:,1]
            split_ind = ((torch.roll(v0, -1, 0) - v0) == 1).nonzero(as_tuple=False)[:,0] + 1
            split_size = torch.cat([split_ind[0].view(1),(torch.roll(split_ind, -1, 0) - split_ind)[:-1]])
            final_size = torch.tensor([torch.numel(v1) - torch.sum(split_size)])
            split_size = torch.cat([split_size, final_size])
            loss = criterion(cls_output, v1)
        return loss, cls_output

model = BertClassifier(model_sber, 3).to(device)

In [27]:
from sklearn.metrics import confusion_matrix

In [28]:
def train(model, iterator, optimizer, scheduler):
    model.train()
    best_loss = 100
    pred = []
    true = []
    total_loss = 0
    correct_predictions = 0
    for x, y in tqdm(iterator):
        optimizer.zero_grad()
        mask = (x != 0).float()
        loss, outputs = model(x, attention_mask=mask, labels=y)
        pred += outputs.detach().numpy().tolist()
        true += y.detach().numpy().tolist()
        #pred = np.argmax(pred, axis = 1) 
        #true = np.argmax(true, axis = 1)
        #print(pred, true)
        #correct_predictions += torch.sum(pred == y)
        total_loss += loss.item()
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        if total_loss / len(iterator) < best_loss:
            best_loss = total_loss / len(iterator)
            torch.save(model.state_dict(), 'saved_weights_tables_16.pt')
    true = np.array(true)
    pred = np.array(pred)
    result_class = np.argmax(pred, axis = 1) 
    label_class = np.argmax(true, axis = 1)
    print(confusion_matrix(label_class, result_class))
    print(f"Train loss {total_loss / len(iterator)}")
    #return correct_predictions.double() / len(iterator)

def evaluate(model, iterator, estate = False):
    model.eval()
    pred = []
    true = []
    correct_predictions = 0

    with torch.no_grad():
        total_loss = 0
        for x, y in tqdm(iterator):
            mask = (x != 0).float()
            loss, outputs = model(x, attention_mask=mask, labels=y)
            total_loss += loss
            true += y.cpu().numpy().tolist()
            pred += outputs.cpu().numpy().tolist()
            #correct_predictions += torch.sum(pred == y)
    true = np.array(true)
    pred = np.array(pred)
    result_class = np.argmax(pred, axis = 1) 
    label_class = np.argmax(true, axis = 1)
    if estate == True:
        print(classification_report(label_class, result_class))
    print(confusion_matrix(label_class, result_class))
    for i, name in enumerate(['Флаг_постоплата', 'Флаг_предоплата',
       'Флаг_помесячно']):
        print(f"{name} roc_auc {roc_auc_score(true[:, i], pred[:, i])}")
    print(f"Evaluate loss {total_loss / len(iterator)}")
    #return correct_predictions.double() / len(iterator)

In [29]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
{'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
{'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}]
EPOCH_NUM = 100
# triangular learning rate, linearly grows untill half of first epoch, then linearly decays 
warmup_steps = 10 ** 3
total_steps = len(train_iterator) * EPOCH_NUM - warmup_steps
optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5, eps=1e-8)
scheduler = get_linear_schedule_with_warmup(optimizer, warmup_steps, total_steps)
# scheduler = WarmupLinearSchedule(optimizer, warmup_steps=warmup_steps, t_total=total_steps)

In [30]:
# sbert_32 ~ 15
# sbert_16 ~ ??
# rus_32 ~ 75
# rus_16 ~ 60

In [31]:
for i in range(EPOCH_NUM):
    print('=' * 50, f"EPOCH {i}", '=' * 50)
    train(model, train_iterator, optimizer, scheduler)
    evaluate(model, dev_iterator, estate = False)

================================================== EPOCH 0 ==================================================


100%|███████████████████████████████████████████| 32/32 [27:00<00:00, 50.64s/it]


[[125  72   4]
 [140  51   4]
 [ 64  40   9]]
Train loss 1.0860347896814346


100%|█████████████████████████████████████████████| 8/8 [00:53<00:00,  6.68s/it]


[[51  0  0]
 [49  0  0]
 [28  0  0]]
Флаг_постоплата roc_auc 0.5884899414311179
Флаг_предоплата roc_auc 0.4900542495479204
Флаг_помесячно roc_auc 0.7517857142857143
Evaluate loss 1.0573433637619019
================================================== EPOCH 1 ==================================================


...

[[196   2   3]
 [  2 193   0]
 [  3   2 108]]
Train loss 0.09323560539633036


100%|█████████████████████████████████████████████| 8/8 [01:08<00:00,  8.56s/it]


[[40  8  3]
 [ 1 48  0]
 [ 2  2 24]]
Флаг_постоплата roc_auc 0.9022154316271963
Флаг_предоплата roc_auc 0.9679669336088865
Флаг_помесячно roc_auc 0.9321428571428572
Evaluate loss 0.6199560761451721
================================================== EPOCH 19 ==================================================


 12%|█████▌                                      | 4/32 [04:29<31:28, 67.46s/it]


KeyboardInterrupt: 

In [32]:
path = 'saved_weights_tables_16.pt'
model.load_state_dict(torch.load(path))
evaluate(model, dev_iterator, estate  = True)

100%|█████████████████████████████████████████████| 8/8 [00:56<00:00,  7.03s/it]

              precision    recall  f1-score   support

           0       0.91      0.78      0.84        51
           1       0.82      0.96      0.89        49
           2       0.89      0.86      0.87        28

    accuracy                           0.87       128
   macro avg       0.87      0.87      0.87       128
weighted avg       0.87      0.87      0.87       128

[[40  8  3]
 [ 2 47  0]
 [ 2  2 24]]
Флаг_постоплата roc_auc 0.9017061370002547
Флаг_предоплата roc_auc 0.968741927150607
Флаг_помесячно roc_auc 0.9307142857142857
Evaluate loss 0.6531900763511658


### создаем лист из предложений

In [ ]:
data_array = []
for i in data_clean.index:
    data_array.extend([data_clean[i]])

In [ ]:
tokens = tokenizer_sber(data_array, padding=True, truncation=True, max_length=48, return_tensors='pt')

In [ ]:
input_ids = tokens.input_ids.numpy() # эмбед-токена
mask = tokens.attention_mask.numpy() # токен-маска
token_type = tokens.token_type_ids.numpy() # тип токена

### скачиваем данные для будущей модели

In [ ]:
save('bert_process/input_ids.npy', input_ids)
save('bert_process/mask.npy', mask)
save('bert_process/token_type.npy', token_type)

### применяем модель - получаем тензор (эмбеддинг) для токенов

In [ ]:
with torch.no_grad():
    model_output = model_sber(**tokens)

In [ ]:
model_output[0][0].shape

In [ ]:
embeddings = model_output.last_hidden_state

In [ ]:
embeddings[0].shape

### сжимаем эмбеддинг по столбцу - получаем эмбеддинг для вектора сглаживанием средних

In [ ]:
mask = tokens['attention_mask'].unsqueeze(-1).expand(embeddings.size()).float()
masked_embeddings = torch.sum(embeddings * mask, 1)
sum_mask = torch.clamp(mask.sum(1), min=1e-9)
sent_embeddings = masked_embeddings / sum_mask

In [ ]:
save('bert_process/bert_output.npy', sent_embeddings.numpy())

### Аналогично выполняем для преобразования тестовой части

In [ ]:
data_array_test = []
for i in range(len(data_clean_test)):
    data_array_test.extend([data_clean_test['Шаблон'].iloc[i]])

In [ ]:
tokens_test = tokenizer_sber(data_array_test, padding=True, truncation=True, max_length=48, return_tensors='pt')

In [ ]:
tokens_test.keys()

In [ ]:
input_ids_test = tokens_test.input_ids.numpy()
mask_test = tokens_test.attention_mask.numpy()
token_type_test = tokens_test.token_type_ids.numpy()

In [ ]:
save('bert_process/input_ids_test.npy', input_ids_test)
save('bert_process/mask_test.npy', mask_test)
save('bert_process/token_type_test.npy', token_type_test)

In [ ]:
with torch.no_grad():
    model_output_test = model_sber(**tokens_test)

In [ ]:
embeddings_test = model_output_test.last_hidden_state
mask = tokens_test['attention_mask'].unsqueeze(-1).expand(embeddings_test.size()).float()
masked_embeddings = torch.sum(embeddings_test * mask, 1)
sum_mask = torch.clamp(mask.sum(1), min=1e-9)
sent_embeddings_test = masked_embeddings / sum_mask

In [ ]:
save('bert_process/bert_output_test.npy', sent_embeddings_test.numpy())